# Memory Managers Demo

This notebook demonstrates how to use the memory managers to read/write shared memory and visualize memory state after each operation.

**Memory Managers:**
- `TaskContextManager` - manages task context at `taskContext` key
- `DatabaseSchemaManager` - manages database schema at `databaseSchema` key  
- `QueryTreeManager` - manages query tree at `queryTree` key
- `NodeHistoryManager` - manages node history at `nodeHistory` key

Each manager is responsible for specific data types and memory locations as defined in TESTING_PLAN.md Layer 1.

In [1]:
# Setup imports and path
import sys
import json
import asyncio
from pathlib import Path
from datetime import datetime

# Add src to path
sys.path.append(str(Path('..') / 'src'))

from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from database_schema_manager import DatabaseSchemaManager
from query_tree_manager import QueryTreeManager
from node_history_manager import NodeHistoryManager
from memory_content_types import (
    TaskStatus, NodeStatus, NodeOperationType,
    TableSchema, ColumnInfo, QueryNode, ExecutionResult, NodeOperation
)

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Helper function to display memory state
def display_memory_state(memory_data, title="Memory State"):
    """
    Display the current state of memory in a formatted way.
    """
    print(f"\n{'='*50}")
    print(f"{title.upper()}")
    print(f"{'='*50}")
    
    if not memory_data:
        print("📝 Memory is empty")
        return
    
    for key, value in memory_data.items():
        print(f"\n🔑 Key: '{key}'")
        print(f"📊 Type: {type(value).__name__}")
        
        if isinstance(value, dict):
            print(f"📁 Dict keys: {list(value.keys())}")
            if len(str(value)) < 200:
                print(f"💾 Content: {json.dumps(value, indent=2, default=str)}")
            else:
                print(f"💾 Content: Large dict with {len(value)} keys (truncated)")
                for k, v in list(value.items())[:3]:
                    print(f"   {k}: {type(v).__name__}")
        elif isinstance(value, list):
            print(f"📋 List length: {len(value)}")
            if value and len(str(value[0])) < 100:
                print(f"💾 First item: {json.dumps(value[0], indent=2, default=str)}")
        else:
            print(f"💾 Content: {str(value)[:200]}")
    
    print(f"\n{'='*50}")

async def show_memory_state(memory, title="Current Memory State"):
    """Async wrapper to get and display memory state."""
    all_data = {}
    # Get all known memory keys
    keys = ['taskContext', 'databaseSchema', 'queryTree', 'nodeHistory']
    
    for key in keys:
        data = await memory.get(key)
        if data is not None:
            all_data[key] = data
    
    display_memory_state(all_data, title)

print("✅ Helper functions defined")

✅ Helper functions defined


## 1. Initialize Shared Memory

Create a shared KeyValueMemory instance that all managers will use.

In [3]:
# Initialize shared memory
memory = KeyValueMemory()

# Show initial empty state
await show_memory_state(memory, "Initial Empty Memory")


INITIAL EMPTY MEMORY
📝 Memory is empty


In [4]:
# Initialize TaskContextManager
task_manager = TaskContextManager(memory)

print("🎯 TaskContextManager Demo")
print("▶️ Initializing task context...")

# Initialize a task
await task_manager.initialize(
    task_id="demo_task_001",
    original_query="Find all schools in California with high SAT scores",
    database_name="california_schools",
    evidence="Focus on schools with average SAT scores above 1200"
)

await show_memory_state(memory, "After Task Initialization")

🎯 TaskContextManager Demo
▶️ Initializing task context...

AFTER TASK INITIALIZATION

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str



In [5]:
print("▶️ Updating task status...")

# Update task status
await task_manager.update_status(TaskStatus.PROCESSING)

# Get and display current context
context = await task_manager.get()
print(f"\n📋 Current Task Context:")
print(f"   Task ID: {context.taskId}")
print(f"   Query: {context.originalQuery}")
print(f"   Database: {context.databaseName}")
print(f"   Status: {context.status.value}")
print(f"   Evidence: {context.evidence}")

await show_memory_state(memory, "After Task Status Update")

▶️ Updating task status...

📋 Current Task Context:
   Task ID: demo_task_001
   Query: Find all schools in California with high SAT scores
   Database: california_schools
   Status: processing
   Evidence: Focus on schools with average SAT scores above 1200

AFTER TASK STATUS UPDATE

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str



## 3. DatabaseSchemaManager Demo

Manages database schema at the `databaseSchema` memory key.

In [6]:
# Initialize DatabaseSchemaManager
schema_manager = DatabaseSchemaManager(memory)
await schema_manager.initialize()

print("🏗️ DatabaseSchemaManager Demo")
print("▶️ Adding database tables...")

# Add schools table
schools_table = TableSchema(
    name="schools",
    columns={
        "CDSCode": ColumnInfo(
            dataType="TEXT", nullable=False, isPrimaryKey=True, isForeignKey=False,
            typicalValues=["01234567890123", "09876543210987"]
        ),
        "School": ColumnInfo(
            dataType="TEXT", nullable=False, isPrimaryKey=False, isForeignKey=False,
            typicalValues=["Washington High School", "Lincoln Elementary"]
        ),
        "County": ColumnInfo(
            dataType="TEXT", nullable=False, isPrimaryKey=False, isForeignKey=False,
            typicalValues=["Los Angeles", "Orange", "San Diego"]
        )
    },
    sampleData=[
        {"CDSCode": "01234567890123", "School": "Washington High School", "County": "Los Angeles"},
        {"CDSCode": "09876543210987", "School": "Lincoln Elementary", "County": "Orange"}
    ],
    metadata={"rowCount": 10000, "lastUpdated": "2024-01-01"}
)

await schema_manager.add_table(schools_table)

await show_memory_state(memory, "After Adding Schools Table")

🏗️ DatabaseSchemaManager Demo
▶️ Adding database tables...

AFTER ADDING SCHOOLS TABLE

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict



In [7]:
print("▶️ Adding SAT scores table with foreign key...")

# Add satscores table with foreign key
satscores_table = TableSchema(
    name="satscores",
    columns={
        "cds": ColumnInfo(
            dataType="TEXT", nullable=False, isPrimaryKey=True, isForeignKey=True,
            references={"table": "schools", "column": "CDSCode"},
            typicalValues=["01234567890123", "09876543210987"]
        ),
        "NumTstTakr": ColumnInfo(
            dataType="INTEGER", nullable=True, isPrimaryKey=False, isForeignKey=False,
            typicalValues=[150, 200, 75]
        ),
        "AvgScrMath": ColumnInfo(
            dataType="INTEGER", nullable=True, isPrimaryKey=False, isForeignKey=False,
            typicalValues=[520, 580, 470]
        ),
        "AvgScrRead": ColumnInfo(
            dataType="INTEGER", nullable=True, isPrimaryKey=False, isForeignKey=False,
            typicalValues=[510, 590, 460]
        )
    },
    sampleData=[
        {"cds": "01234567890123", "NumTstTakr": 150, "AvgScrMath": 520, "AvgScrRead": 510},
        {"cds": "09876543210987", "NumTstTakr": 200, "AvgScrMath": 580, "AvgScrRead": 590}
    ],
    metadata={"rowCount": 8500, "testYear": 2023}
)

await schema_manager.add_table(satscores_table)

# Query schema information
print(f"\n📊 Schema Summary:")
table_names = await schema_manager.get_table_names()
print(f"   Tables: {table_names}")

foreign_keys = await schema_manager.get_foreign_keys("satscores")
print(f"   Foreign keys in satscores: {foreign_keys}")

await show_memory_state(memory, "After Adding SAT Scores Table")

▶️ Adding SAT scores table with foreign key...

📊 Schema Summary:
   Tables: ['schools', 'satscores']
   Foreign keys in satscores: [{'column': 'cds', 'references_table': 'schools', 'references_column': 'CDSCode'}]

AFTER ADDING SAT SCORES TABLE

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict



## 4. QueryTreeManager Demo

Manages query tree at the `queryTree` memory key.

In [8]:
# Initialize QueryTreeManager
tree_manager = QueryTreeManager(memory)

print("🌳 QueryTreeManager Demo")
print("▶️ Initializing query tree...")

# Initialize query tree with root node
root_id = await tree_manager.initialize(
    root_intent="Find all schools in California with high SAT scores",
    evidence="Focus on schools with average SAT scores above 1200"
)

print(f"\n🌟 Root node created: {root_id}")

await show_memory_state(memory, "After Query Tree Initialization")

🌳 QueryTreeManager Demo
▶️ Initializing query tree...

🌟 Root node created: root

AFTER QUERY TREE INITIALIZATION

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict



In [9]:
print("▶️ Adding child nodes...")

# Add child nodes for decomposed query
child1 = QueryNode(
    nodeId="node_schools",
    intent="Get all schools in California",
    parentId=root_id,
    status=NodeStatus.CREATED
)

child2 = QueryNode(
    nodeId="node_sat_scores", 
    intent="Filter schools by high SAT scores",
    parentId=root_id,
    status=NodeStatus.CREATED
)

await tree_manager.add_node(child1, root_id)
await tree_manager.add_node(child2, root_id)

# Set current working node
await tree_manager.set_current_node_id("node_schools")

print(f"\n📍 Current node: {await tree_manager.get_current_node_id()}")
print(f"🔗 Root children: {[c.nodeId for c in await tree_manager.get_children(root_id)]}")

await show_memory_state(memory, "After Adding Child Nodes")

▶️ Adding child nodes...

📍 Current node: node_schools
🔗 Root children: ['node_schools', 'node_sat_scores']

AFTER ADDING CHILD NODES

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict



In [10]:
print("▶️ Adding SQL and execution results...")

# Add SQL to first child node
await tree_manager.update_node_sql(
    "node_schools", 
    "SELECT CDSCode, School, County FROM schools WHERE State = 'CA'"
)

# Simulate execution result
execution_result = ExecutionResult(
    data=[
        {"CDSCode": "01234567890123", "School": "Washington High School", "County": "Los Angeles"},
        {"CDSCode": "09876543210987", "School": "Lincoln Elementary", "County": "Orange"}
    ],
    rowCount=2,
    error=None
)

await tree_manager.update_node_result("node_schools", execution_result, success=True)

# Check total nodes
query_tree = await tree_manager.get_tree()
print(f"🔗 All nodes: {[c for c in query_tree['nodes']]}")

# Check node status
node = await tree_manager.get_node("node_schools")
print(f"\n✅ Node status: {node.status.value}")
print(f"📊 Execution result: {node.evaluation.get('execution_result', {}).get('rowCount')} rows")

await show_memory_state(memory, "After Adding SQL and Results")

▶️ Adding SQL and execution results...
🔗 All nodes: ['root', 'node_schools', 'node_sat_scores']

✅ Node status: executed_success
📊 Execution result: None rows

AFTER ADDING SQL AND RESULTS

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict



## 5. NodeHistoryManager Demo

Manages node history at the `nodeHistory` memory key.

In [11]:
# Initialize NodeHistoryManager
history_manager = NodeHistoryManager(memory)
await history_manager.initialize()

print("📚 NodeHistoryManager Demo - Simple Use Case")
print("▶️ Recording node operation history...")

# Create a simple node for history tracking
simple_node = QueryNode(
    nodeId="node_schools",
    intent="Get all schools in California",
    status=NodeStatus.CREATED,
    evidence="Need to identify all California schools"
)

# Record node creation
await history_manager.record_create(simple_node)
print("✅ Recorded node creation")

await show_memory_state(memory, "After Recording Node Creation")

📚 NodeHistoryManager Demo - Simple Use Case
▶️ Recording node operation history...
✅ Recorded node creation

AFTER RECORDING NODE CREATION

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict

🔑 Key: 'nodeHistory'
📊 Type: list
📋 List length: 1



In [12]:
# Simple workflow: Record SQL generation and execution
print("\n▶️ Recording SQL generation...")

# Update node with SQL
simple_node.status = NodeStatus.SQL_GENERATED
simple_node.generation = {
    "sql": "SELECT CDSCode, School, County FROM schools WHERE State = 'CA'",
    "sql_type": "SELECT",
    "confidence": 0.92
}

await history_manager.record_generate_sql(simple_node)
print("✅ Recorded SQL generation")

# Simulate execution with results
print("\n▶️ Recording execution result...")
simple_node.status = NodeStatus.EXECUTED_SUCCESS
simple_node.evaluation = {
    "execution_result": {
        "data": [
            {"CDSCode": "01234567890123", "School": "Washington High", "County": "Los Angeles"},
            {"CDSCode": "09876543210987", "School": "Lincoln Elementary", "County": "Orange"}
        ],
        "rowCount": 2
    },
    "success": True
}

await history_manager.record_execute(simple_node)
print("✅ Recorded successful execution")

# Get simple analytics
print("\n📊 Simple History Analysis:")
node_ops = await history_manager.get_node_operations("node_schools")
print(f"   Total operations: {len(node_ops)}")
for i, op in enumerate(node_ops):
    print(f"   {i+1}. {op.operation.value} at {op.timestamp[:19]}")

# Get execution summary
summary = await history_manager.get_history_summary()
print(f"\n📈 Quick Stats:")
print(f"   Total nodes: {summary['unique_nodes']}")
print(f"   SQL generations: {summary['sql_generation_count']}")
print(f"   Executions: {summary['execution_stats']['total_executions']}")
print(f"   Success rate: {summary['execution_stats']['success_rate']:.0%}")


▶️ Recording SQL generation...
✅ Recorded SQL generation

▶️ Recording execution result...
✅ Recorded successful execution

📊 Simple History Analysis:
   Total operations: 3
   1. create at 2025-05-29T16:50:34
   2. generate_sql at 2025-05-29T16:50:34
   3. execute at 2025-05-29T16:50:34

📈 Quick Stats:
   Total nodes: 1
   SQL generations: 1
   Executions: 1
   Success rate: 100%


In [13]:
# Demonstrate error handling and retry
print("\n▶️ Simulating error and retry...")

# Create another node that will fail first
retry_node = QueryNode(
    nodeId="node_sat_scores",
    intent="Get SAT scores for schools",
    status=NodeStatus.CREATED
)

await history_manager.record_create(retry_node)

# First attempt - fails
retry_node.status = NodeStatus.SQL_GENERATED
retry_node.generation = {"sql": "SELECT * FROM wrong_table", "confidence": 0.7}
await history_manager.record_generate_sql(retry_node)

retry_node.status = NodeStatus.EXECUTED_FAILED
await history_manager.record_execute(retry_node, error="Table 'wrong_table' not found")
print("❌ First attempt failed")

# Revise and retry
retry_node.status = NodeStatus.REVISED
await history_manager.record_revise(retry_node, reason="Fix table name")

retry_node.generation = {"sql": "SELECT * FROM satscores", "confidence": 0.95}
await history_manager.record_generate_sql(retry_node)

retry_node.status = NodeStatus.EXECUTED_SUCCESS
retry_node.evaluation = {"execution_result": {"rowCount": 100}, "success": True}
await history_manager.record_execute(retry_node)
print("✅ Retry succeeded")

# Show retry analysis
print("\n🔄 Retry Analysis:")
attempts = await history_manager.get_node_attempts_summary("node_sat_scores")
print(f"   Total attempts: {attempts['total_attempts']}")
print(f"   Final status: {attempts['final_status']}")

await show_memory_state(memory, "After Complete Workflow")


▶️ Simulating error and retry...
❌ First attempt failed
✅ Retry succeeded

🔄 Retry Analysis:
   Total attempts: 2
   Final status: executed_success

AFTER COMPLETE WORKFLOW

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict

🔑 Key: 'nodeHistory'
📊 Type: list
📋 List length: 9



In [14]:
# ## 6. Memory State Analysis
#
# Let's analyze what's in each memory location and how managers interact with shared memory.

In [15]:
print("🔍 Memory State Analysis")
print("="*60)

# Analyze each memory location
print("\n1️⃣ TASK CONTEXT (taskContext key):")
task_data = await memory.get("taskContext")
if task_data:
    print(f"   📌 Task ID: {task_data['taskId']}")
    print(f"   📋 Query: {task_data['originalQuery'][:50]}...")
    print(f"   🏗️ Database: {task_data['databaseName']}")
    print(f"   🔄 Status: {task_data['status']}")
    print(f"   💡 Evidence: {task_data.get('evidence', 'None')[:50]}...")

print("\n2️⃣ DATABASE SCHEMA (databaseSchema key):")
schema_data = await memory.get("databaseSchema")
if schema_data:
    tables = schema_data.get('tables', {})
    print(f"   📊 Number of tables: {len(tables)}")
    for table_name, table_info in tables.items():
        columns = table_info.get('columns', {})
        sample_count = len(table_info.get('sampleData', []))
        print(f"   📁 {table_name}: {len(columns)} columns, {sample_count} sample rows")

print("\n3️⃣ QUERY TREE (queryTree key):")
tree_data = await memory.get("queryTree")
if tree_data:
    nodes = tree_data.get('nodes', {})
    print(f"   🌳 Root ID: {tree_data.get('rootId')}")
    print(f"   📍 Current Node: {tree_data.get('currentNodeId')}")
    print(f"   🔢 Total nodes: {len(nodes)}")
    for node_id, node_info in nodes.items():
        status = node_info.get('status', 'unknown')
        intent = node_info.get('intent', 'No intent')[:30]
        children = len(node_info.get('childIds', []))
        print(f"   📄 {node_id}: {status} | {intent}... | {children} children")

print("\n4️⃣ NODE HISTORY (nodeHistory key) - ENHANCED:")
history_data = await memory.get("nodeHistory")
if history_data:
    print(f"   📚 Total operations: {len(history_data)}")
    
    # Group by operation type
    op_counts = {}
    node_counts = {}
    essential_info_examples = []
    
    for op in history_data:
        op_type = op.get('operation', 'unknown')
        node_id = op.get('nodeId', 'unknown')
        op_counts[op_type] = op_counts.get(op_type, 0) + 1
        node_counts[node_id] = node_counts.get(node_id, 0) + 1
        
        # Check for essential information filtering
        data = op.get('data', {})
        if 'schema_linking' in data:
            schema = data['schema_linking']
            # Check if verbose fields were filtered out
            if 'verbose_explanation' not in str(schema):
                essential_info_examples.append(f"{op_type}: verbose content filtered")
    
    print(f"   🔧 Operations by type: {dict(op_counts)}")
    print(f"   📊 Operations by node: {dict(node_counts)}")
    
    # Show memory optimization
    total_chars = sum(len(str(op)) for op in history_data)
    avg_chars = total_chars // len(history_data) if history_data else 0
    print(f"   💾 Memory usage: {total_chars:,} chars (~{avg_chars} per operation)")
    print(f"   ✅ Essential info filtering: {len(essential_info_examples)} examples")
    
    # Show sample operation structure
    if history_data:
        sample_op = history_data[0]
        sample_data = sample_op.get('data', {})
        essential_fields = [k for k in sample_data.keys() if k in ['nodeId', 'intent', 'status', 'evidence', 'sql']]
        print(f"   🔍 Sample operation fields: {essential_fields}")

print("\n" + "="*60)

🔍 Memory State Analysis

1️⃣ TASK CONTEXT (taskContext key):
   📌 Task ID: demo_task_001
   📋 Query: Find all schools in California with high SAT score...
   🏗️ Database: california_schools
   🔄 Status: processing
   💡 Evidence: Focus on schools with average SAT scores above 120...

2️⃣ DATABASE SCHEMA (databaseSchema key):
   📊 Number of tables: 2
   📁 schools: 3 columns, 2 sample rows
   📁 satscores: 4 columns, 2 sample rows

3️⃣ QUERY TREE (queryTree key):
   🌳 Root ID: root
   📍 Current Node: node_schools
   🔢 Total nodes: 3
   📄 root: created | Find all schools in California... | 2 children
   📄 node_schools: executed_success | Get all schools in California... | 0 children
   📄 node_sat_scores: created | Filter schools by high SAT sco... | 0 children

4️⃣ NODE HISTORY (nodeHistory key) - ENHANCED:
   📚 Total operations: 9
   🔧 Operations by type: {'create': 2, 'generate_sql': 3, 'execute': 3, 'revise': 1}
   📊 Operations by node: {'node_schools': 3, 'node_sat_scores': 6}
   💾 Memo

In [17]:
print("🔄 Manager Interaction Patterns")
print("="*50)

print("\n▶️ Simulating a complete workflow...")

# 1. Task manager updates status
await task_manager.update_status(TaskStatus.COMPLETED)
print("✅ Task marked as completed")

# 2. Query tree manager adds more analysis
await tree_manager.update_node_evaluation("node_schools", {
    "result_quality": "excellent",
    "answers_intent": True,
    "confidence": 0.95
})
print("✅ Added evaluation to node")

# 3. History manager records completion - create the node first
completion_node = QueryNode(
    nodeId="node_sat_scores",
    intent="Get SAT scores for schools",
    status=NodeStatus.EXECUTED_SUCCESS,
    generation={
        "sql": "SELECT s.School, s.County, sat.AvgScrMath FROM schools s JOIN satscores sat ON s.CDSCode = sat.cds WHERE sat.AvgScrMath > 1200"
    },
    evaluation={
        "execution_result": {"rowCount": 25, "data": "[...25 high-scoring schools...]"},
        "success": True
    }
)

await history_manager.record_execute(completion_node)
print("✅ Recorded final execution")

# 4. Get cross-manager statistics
print("\n📊 Cross-Manager Summary:")
task_context = await task_manager.get()
tree_stats = await tree_manager.get_tree_stats()
all_operations = await history_manager.get_all_operations()
schema_summary = await schema_manager.get_schema_summary()

print(f"   🎯 Task: {task_context.status.value} - {task_context.originalQuery[:40]}...")
print(f"   🏗️ Schema: {schema_summary['table_count']} tables, {schema_summary['total_columns']} columns")
print(f"   🌳 Tree: {tree_stats['total_nodes']} nodes, depth {tree_stats['depth']}")
print(f"   📚 History: {len(all_operations)} operations across workflow")

await show_memory_state(memory, "Final Memory State")

🔄 Manager Interaction Patterns

▶️ Simulating a complete workflow...
✅ Task marked as completed
✅ Added evaluation to node
✅ Recorded final execution

📊 Cross-Manager Summary:
   🎯 Task: completed - Find all schools in California with high...
   🏗️ Schema: 2 tables, 7 columns
   🌳 Tree: 3 nodes, depth 1
   📚 History: 11 operations across workflow

FINAL MEMORY STATE

🔑 Key: 'taskContext'
📊 Type: dict
📁 Dict keys: ['taskId', 'originalQuery', 'databaseName', 'startTime', 'status', 'evidence']
💾 Content: Large dict with 6 keys (truncated)
   taskId: str
   originalQuery: str
   databaseName: str

🔑 Key: 'databaseSchema'
📊 Type: dict
📁 Dict keys: ['tables', 'metadata']
💾 Content: Large dict with 2 keys (truncated)
   tables: dict
   metadata: dict

🔑 Key: 'queryTree'
📊 Type: dict
📁 Dict keys: ['rootId', 'currentNodeId', 'nodes']
💾 Content: Large dict with 3 keys (truncated)
   rootId: str
   currentNodeId: str
   nodes: dict

🔑 Key: 'nodeHistory'
📊 Type: list
📋 List length: 11



## 8. Memory Key Verification

Verify that each manager uses the correct memory keys as specified in TESTING_PLAN.md Layer 1.

In [18]:
print("🔑 Memory Key Verification")
print("="*40)

# Check each expected memory key
expected_keys = {
    "taskContext": "TaskContextManager",
    "databaseSchema": "DatabaseSchemaManager", 
    "queryTree": "QueryTreeManager",
    "nodeHistory": "NodeHistoryManager"
}

print("\n📋 Memory Key Usage:")
for key, manager_name in expected_keys.items():
    data = await memory.get(key)
    if data is not None:
        data_type = type(data).__name__
        size = len(str(data))
        print(f"   ✅ {key:<15} | {manager_name:<22} | {data_type:<8} | {size:>6} chars")
    else:
        print(f"   ❌ {key:<15} | {manager_name:<22} | Missing")

# Verify no unexpected keys
print("\n🔍 Checking for unexpected keys...")
all_keys = set()
# Note: KeyValueMemory doesn't have a list_keys method, so we check known keys
for key in ["taskContext", "databaseSchema", "queryTree", "nodeHistory"]:
    if await memory.get(key) is not None:
        all_keys.add(key)

unexpected = all_keys - set(expected_keys.keys())
if unexpected:
    print(f"   ⚠️ Unexpected keys found: {list(unexpected)}")
else:
    print(f"   ✅ All keys match expectations")

print(f"\n📊 Total memory locations used: {len(all_keys)}")
print(f"📊 Expected memory locations: {len(expected_keys)}")
print(f"✅ Memory key compliance: {'PASS' if all_keys == set(expected_keys.keys()) else 'FAIL'}")

🔑 Memory Key Verification

📋 Memory Key Usage:
   ✅ taskContext     | TaskContextManager     | dict     |    270 chars
   ✅ databaseSchema  | DatabaseSchemaManager  | dict     |   1773 chars
   ✅ queryTree       | QueryTreeManager       | dict     |   1202 chars
   ✅ nodeHistory     | NodeHistoryManager     | list     |   4335 chars

🔍 Checking for unexpected keys...
   ✅ All keys match expectations

📊 Total memory locations used: 4
📊 Expected memory locations: 4
✅ Memory key compliance: PASS


## 9. Cleanup and Summary

Clear memory and summarize the demo.

In [19]:
print("🧹 Cleanup and Summary")
print("="*30)

# Clear memory
await memory.clear()
print("✅ Memory cleared")

# Verify cleanup
await show_memory_state(memory, "Memory After Cleanup")

print("\n📝 Demo Summary:")
print("   ✅ TaskContextManager: Manages task context at 'taskContext' key")
print("   ✅ DatabaseSchemaManager: Manages schema at 'databaseSchema' key")
print("   ✅ QueryTreeManager: Manages query tree at 'queryTree' key")
print("   ✅ NodeHistoryManager: Tracks operation history at 'nodeHistory' key")

print("\n🎯 Key Learnings:")
print("   🔑 Each manager uses a specific memory key")
print("   🔄 Managers share the same KeyValueMemory instance")
print("   📊 Memory state is persistent across manager operations")
print("   🔍 Memory contents can be inspected at any time")
print("   ⚡ Operations are immediate and reflected in memory")

print("\n🚀 NodeHistoryManager Features:")
print("   📚 Tracks all node operations chronologically")
print("   🔄 Supports error tracking and retry analysis")
print("   📊 Provides execution statistics and success rates")
print("   💾 Automatically filters verbose content for efficiency")
print("   🔍 Enables debugging through operation history")

print("\n🎉 Demo completed successfully!")
print("📖 For advanced NodeHistoryManager features, see 01-manager-nodehistory.ipynb")

🧹 Cleanup and Summary
✅ Memory cleared

MEMORY AFTER CLEANUP
📝 Memory is empty

📝 Demo Summary:
   ✅ TaskContextManager: Manages task context at 'taskContext' key
   ✅ DatabaseSchemaManager: Manages schema at 'databaseSchema' key
   ✅ QueryTreeManager: Manages query tree at 'queryTree' key
   ✅ NodeHistoryManager: Tracks operation history at 'nodeHistory' key

🎯 Key Learnings:
   🔑 Each manager uses a specific memory key
   🔄 Managers share the same KeyValueMemory instance
   📊 Memory state is persistent across manager operations
   🔍 Memory contents can be inspected at any time
   ⚡ Operations are immediate and reflected in memory

🚀 NodeHistoryManager Features:
   📚 Tracks all node operations chronologically
   🔄 Supports error tracking and retry analysis
   📊 Provides execution statistics and success rates
   💾 Automatically filters verbose content for efficiency
   🔍 Enables debugging through operation history

🎉 Demo completed successfully!
📖 For advanced NodeHistoryManager featur